***Installing Packages Needed for This Notebook and/or Beyond***

In [1]:
%pip install --upgrade pip
%pip install numpy
%pip install scipy
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


***Import useful packages***

In [2]:
from scipy.fft import fft, ifft, fftfreq
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.sparse.linalg import spsolve

# Exercise 1.2 Stopping light

## Analytical solution for free evolution

For the free evolution of a Gaussian wavepacket, the analytical solution is known and given by:
$ \Psi(x,t)=\mathcal{N}\sqrt{\frac{a^2}{a^2+2it}}e^{i(k_0x-k_0^2t/2)}e^{-(x-x_0-k_0t)^2/(a^2+2it)}$. The normalization factor is $\mathcal{N}=(\pi a^2/2)^{-1/4}$.

In [ ]:
def phi_exact(x, t=0, a=1, k_0=5, x_0=-10):
    """
    Analytical solution of free evolution of Gaussian wavepacket.

    Parameters
    ----------
    x : float or array_like
        As we only use numpy functions, `exact_wavepacket` is 
        vectorized and can accept numbers as well as arrays as
        `x` input.

    Returns
    -------
    array_like
        Exact solution for wavepacket at time t.
    
    """

    N = (np.pi * a**2 / 2)**(-1/4)
    factor_1 = np.sqrt(a**2 / (a**2 + 2j * t))
    exp_1 = np.exp(1j * (k_0 * x - k_0**2 * t / 2))
    gaussian_exp = np.exp(-(x - x_0 - k_0 * t)**2 / (a**2 + 2j * t))
    wavepacket = N * factor_1 * exp_1 * gaussian_exp
    return wavepacket

In [ ]:
# The parameters for the simulation
x_steps = 1000
x_values, dx = np.linspace(-20, 50, x_steps, retstep=True)

t_final = 3
pot_free = np.vectorize(lambda x: 0)
phi0 = phi_exact(x_values, t=0)

We define the average position, group velocity, spread (as FWHM) and norm of a generic wavepacket. The velocity of the wavepacket is given by 
$v(t)=-\int_{-\infty}^{\infty} dx Im \Psi \nabla \Psi^*$.

In [ ]:
def position(wavepacket, x_values, dx):
    # <x> = -int x * |Psi(x)|^2 dx
    pos = sum(x_values * abs(wavepacket) ** 2 * dx)/norm(wavepacket, dx)
    return pos

def velocity(wavepacket, dx):
    wavepacket_mean = (wavepacket[1:] + wavepacket[:-1]) / 2.
    wavepacket_prime = np.diff(wavepacket).conj() / dx
    probability_current = -(wavepacket_mean * wavepacket_prime).imag
    average_velocity = sum(probability_current * dx)
    return np.real(average_velocity)

# FWHM
def spread(wavepacket, x_values):
    half_max = max(abs(wavepacket))/2
    diff = abs(wavepacket) - half_max
    node_positions = [i*j < 0 for i, j in zip(diff, diff[1:])]
    nodes = x_values[1:][node_positions]
    return nodes[-1] - nodes[0]

def norm(wavepacket, dx):
    return sum(abs(wavepacket)**2 * dx)

## Hamiltonian
Here you should implement a discretized Hamiltonian operator that you will use in the rest of the exercise. For this use the approximation:

$\frac{d^2f}{dx^2}(x) \approx \frac{f(x+\Delta x) - 2f(x) + f(x-\Delta x)}{(\Delta x)^2}$

to approximate the kinetic term in the Hamiltonian:
$H = -\frac{\hbar^2}{2m}\frac{d^2}{dx^2} + V$

In [ ]:
def hamiltonian(pot, x_values, dx):
    """
    Compute the discretized Hamiltonian for the potential `pot`.

    Parameters
    ----------
    pot : function
        A (vectorized) function describing the shape of the potential.
    x_values : array_like
        The x coordinates of the discretized wave function.
    dx : float
        The difference between discretized x coordinates.

    Returns
    -------
    H : array_like
        The discretized Hamiltonian in matrix form.
    """

    # TODO: Compute the Hamiltonian H.
    return scipy.sparse.csc_matrix(H) # Return a sparse version of the Hamiltonian

In [ ]:
# Helper function for making some nice plots
def make_plot(phi0, t_final, n_steps, pot, method, title=""):
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=False, figsize=(12, 5))
    fig.suptitle(title)
    ax1.plot(x_values, abs(phi0), label=r"$\phi(t=0)$")
    
    phi_num, positions, velocities, spreads, norms = method(phi0, t_final, n_steps=n_steps, pot=pot)
    ax1.plot(x_values, abs(phi_num), label=f"$\phi(t={{{t_final}}})$ numeric")
    
    ax1.plot(x_values, abs(phi_exact(x_values, t_final)), "--", label=f"$\phi(t={{{t_final}}})$ exact")
    ax1.set_xlabel(r"x")
    ax1.set_ylabel(r"$|\phi(x)|^2$")
    ax1.legend()

    ts = np.linspace(0, t_final, n_steps)
    ax2.plot(ts, positions, label="Position")
    ax2.plot(ts, velocities, label="Velocity")
    ax2.plot(ts, spreads, label="Spread")
    ax2.set_xlabel(r"t")
    ax2.plot(ts, norms, label="Norm")
    ax2.legend()
    fig.tight_layout()

### Plus: Euler forward method
The Euler forward method comes from the first order expansion of the time evolution operator:

$
|\Psi(t + \Delta t)\rangle \approx |\Psi(t)\rangle - \frac{i \Delta t}{\hbar}H|\Psi(t)\rangle
$

As discussed in the lecture, the Euler forward method is not only numerically unstable, but also violates conservation of norm of the wavefunction. To see this, we implement the Euler forward method. Here, you don't have to do anything yourself (except for the implementation of the Hamiltonian above), just have a look at the plot!

In [ ]:
def evolve_euler_forward(phi, t_final, n_steps, pot):
    """
    Perform time evolution using the forward Euler method.

    Parameters
    ----------
    phi : array_like
        The wave function at time t=0.
    t_final : float
        The final time of the time evolution.
    n_steps : int
        The number of steps used for time evolution.
    pot : function
        A (vectorized) function describing the shape of the potential.

    Returns
    -------
    phi : array_like
        The wave function at time t=t_final.
    positions : array_like
        The position of the wave function at intermediate times.
    velocities : array_like
        The velocity of the wave function at intermediate times.
    spreads : array_like
        The spread of the wave function at intermediate times.
    norms : array_like
        The norm of the wave function at intermediate times.
    """

    dt = t_final / n_steps

    id = scipy.sparse.identity(len(x_values))
    H = hamiltonian(pot, x_values, dx)
    M = id - 1j*dt*H

    positions = []
    velocities = []
    spreads = []
    norms = []
    for i in range(n_steps):
        phi_new = M @ phi
        phi = phi_new
        
        positions.append(position(phi, x_values, dx))
        velocities.append(velocity(phi, dx))
        spreads.append(spread(phi, x_values))
        norms.append(norm(phi, dx))

    return phi, positions, velocities, spreads, norms

In [ ]:
make_plot(phi0, 2.0, 1_000, pot_free, method=evolve_euler_forward, title="Forward euler")

## Part 2: numerical free time evolution

### (a) Spectral method
The simplest and most direct method is by explicitly evaluating the propagator:

$U(t) = \exp[-iH/\hbar]$,

where $\exp$ is the matrix exponential and can be evaluated using either `np.linalg.expm` or `scipy.sparse.linalg.expm`.

In [ ]:
def evolve_spectral(phi, t_final, n_steps, pot):
    """
    Perform time evolution using the spectral method.

    Parameters
    ----------
    phi : array_like
        The wave function at time t=0.
    t_final : float
        The final time of the time evolution.
    n_steps : int
        The number of steps used for time evolution.
    pot : function
        A (vectorized) function describing the shape of the potential.

    Returns
    -------
    phi : array_like
        The wave function at time t=t_final.
    positions : array_like
        The position of the wave function at intermediate times.
    velocities : array_like
        The velocity of the wave function at intermediate times.
    spreads : array_like
        The spread of the wave function at intermediate times.
    norms : array_like
        The norm of the wave function at intermediate times.
    """

    # TODO: Implement the spectral method.
    pass

In [ ]:
make_plot(phi0, 3, 500, pot_free, method=evolve_spectral, title="Spectral method")

### (b) Unitary direct numerical integration scheme
Implement the unitary direct numerical integration scheme:

$
\left(\mathbb{I} + \frac{i\Delta t}{2\hbar}H\right)|\Psi(t+\Delta t)\rangle = \left(\mathbb{I} - \frac{i\Delta t}{2\hbar}H\right)|\Psi(t)\rangle.
$
You can use `scipy.sparse.linalg.spsolve` to solve the linear system (when working with sparse matrices).

To see that this evolution is unitary we note that the evolution operator is given by

$M = \left(\mathbb{I} + \frac{i\Delta t}{2\hbar}H\right)^{-1} \left(\mathbb{I} - \frac{i\Delta t}{2\hbar}H\right).$

Check that $M^{\dagger}M=\mathbb{1}$ and $M M^{\dagger}$ (Part 1 of the exercise).

In [ ]:
def evolve_unitarily(phi, t_final, n_steps, pot):
    """
    Perform time evolution using the unitary integration method.

    Parameters
    ----------
    phi : array_like
        The wave function at time t=0.
    t_final : float
        The final time of the time evolution.
    n_steps : int
        The number of steps used for time evolution.
    pot : function
        A (vectorized) function describing the shape of the potential.
 
    Returns
    -------
    phi : array_like
        The wave function at time t=t_final.
    positions : array_like
        The position of the wave function at intermediate times.
    velocities : array_like
        The velocity of the wave function at intermediate times.
    spreads : array_like
        The spread of the wave function at intermediate times.
    norms : array_like
        The norm of the wave function at intermediate times.
    """
    
    # TODO: Implement the unitary evolution operator.
    pass

In [ ]:
make_plot(phi0, t_final, 1_000, pot_free, method=evolve_unitarily, title="Unitary integrator")

### (c) Split operator method
Making use of the fact that the kinetic part $T$ of the Hamiltonian is diagonal in momentum space and the potential part $V$ diagonal in position space, one can define the split operator method. In it, the evolution by a single timestep is split into two half-timesteps by the potential operator (in position space) and a full timestep by the momentum operator (in momentum space):

$
e^{-i\Delta t H/\hbar}\approx e^{-i\Delta t V/2\hbar}e^{-i\Delta t T/\hbar} e^{-i\Delta t V/2\hbar}
$

The full time evolution operator up to time $t=N\Delta t$ is then given by:

$
e^{-i t H/\hbar} \approx e^{-i\Delta t V/2\hbar}\left[e^{-i\Delta t T/\hbar} e^{-i\Delta t V/\hbar}\right]^{N-1}e^{-i\Delta t T/\hbar} e^{-i\Delta t V/2\hbar}.
$

The basis changes between position and momentum space can be done using a discrete Fourier transorm (e.g. `scipy.fftpack.fft` and `scipy.fftpack.ifft`).

In [ ]:
def evolve_split_operator(phi, t_final, n_steps, pot):
    """
    Perform time evolution using the split operator method.

    Parameters
    ----------
    phi : array_like
        The wave function at time t=0.
    t_final : float
        The final time of the time evolution.
    n_steps : int
        The number of steps used for time evolution.
    pot : function
        A (vectorized) function describing the shape of the potential.

    Returns
    -------
    phi : array_like
        The wave function at time t=t_final.
    positions : array_like
        The position of the wave function at intermediate times.
    velocities : array_like
        The velocity of the wave function at intermediate times.
    spreads : array_like
        The spread of the wave function at intermediate times.
    norms : array_like
        The norm of the wave function at intermediate times.
    """

    # TODO: Implement the split operator method.
    pass

In [ ]:
make_plot(phi0, t_final, 100, pot_free, method=evolve_split_operator, title="Split operator")

## Part 3: Tilted Wall

Let us introduce a non-zero potential: a tilted wall with angle $\theta$. You will see that the wavepacket will rebound at the tilted wall. For this, plot position and velocity of the wavepacket as a function of time for different angles $\theta$ (you can use any of the numerical time evolution methods implemented above).

In [ ]:
def pot_wall(theta_deg):
    m = np.tan(theta_deg * np.pi / 180.)
    return np.vectorize(lambda x: max(0., m*x))

In [ ]:
# TODO: Plot the position and velocity versus time for the tilted wall at various angles.